https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part9.html

In [ ]:
!pip install fugashi
!pip install ipadic
!pip install mecab-python3
!pip install ginza
!pip install -U ginza ja-ginza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.1 MB 131 kB/s 


In [ ]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers
!pip install sentence_transformers
!pip install numpy 
!pip install spacy 

In [ ]:
import transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers import models

# SentenceBERTはトークナイザにBertTokenizerを決め打ちで使うので、ここで修正
transformers.BertTokenizer = transformers.BertJapaneseTokenizer

transformer = models.Transformer('cl-tohoku/bert-base-japanese-whole-word-masking')
pooling = models.Pooling(transformer.get_word_embedding_dimension(),
                         pooling_mode_mean_tokens=True,
                         pooling_mode_cls_token=False, 
                         pooling_mode_max_tokens=False)
model=SentenceTransformer(modules=[transformer,pooling])

#文章を固定長ベクトルに変換
sentences=["吾輩は猫である","本日は晴天なり"]
embeddings=model.encode(sentences)

for i,embedding in enumerate(embeddings):
    print("[%d]:%s"%(i,embedding.shape,))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[0]:(768,)
[1]:(768,)


In [ ]:
#学習データセットの準備
!git clone https://github.com/STAIR-Lab-CIT/STAIR-captions
!tar zxvf STAIR-captions/stair_captions_v1.2.tar.gz
!ls -lh *.json

import json
with open("stair_captions_v1.2_val.json", "r") as f:
  json_data_val = json.load(f)
with open("stair_captions_v1.2_train.json", "r") as f:
  json_data_train = json.load(f)

fatal: destination path 'STAIR-captions' already exists and is not an empty directory.
stair_captions_v1.2_train.json
stair_captions_v1.2_train_tokenized.json
stair_captions_v1.2_val.json
stair_captions_v1.2_val_tokenized.json
-rw-rw-r-- 1 1005 1006  67M Jan 26  2018 stair_captions_v1.2_train.json
-rw-rw-r-- 1 1005 1006 106M Jan 26  2018 stair_captions_v1.2_train_tokenized.json
-rw-rw-r-- 1 1005 1006  33M Jan 26  2018 stair_captions_v1.2_val.json
-rw-rw-r-- 1 1005 1006  52M Jan 26  2018 stair_captions_v1.2_val_tokenized.json


In [ ]:
#dataset は画像ID をキー、画像に付与された5つのキャプションの(ID, 文字列)のタプルのリストを値に持つ辞書
#ids はキャプションIDのリスト
#captions はキャプション文字列のリスト

dataset={}
ids,captions=[],[]

def build_dataset(dataset,json_data):
    num_samples=len(json_data["annotations"])
    for i in range(num_samples):
        anno=json_data["annotations"][i]
        image_id=anno["image_id"]
        image_captions = dataset.get(image_id, [])
        image_captions.append((anno["id"], anno["caption"]))
        ids.append(anno["id"])
        captions.append(anno["caption"])
        dataset[image_id] = image_captions

build_dataset(dataset, json_data_val)
build_dataset(dataset, json_data_train)

#キャプションID と ids, captions のインデックスの変換テーブルを作る
id2idx = {id:idx for idx, id in enumerate(ids)}

In [ ]:
len(captions)

616435

In [ ]:
import numpy as np
import spacy
import pkg_resources, imp

#キャプション文字列を、単語ベクトルの平均を取ることで文章ベクトルに変換

imp.reload(pkg_resources)
nlp=spacy.load("ja_ginza")

vectors=[]
i=0
for caption in captions:
    i+=1
    if i %5000==0:
      print(i)
    doc=nlp(caption,disable=["ner"])
    vectors.append(doc.vector)
del nlp

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000
470000
475000
480000
485000
490000
495000
500000
505000
510000
515000
520000
525000
530000
535000
540000
545000
550000
555000
560000
565000
570000
575000
580000
585000
590000
595000
600000
605000
610000
615000


In [ ]:
from numpy import dot 
from numpy.linalg import norm 
from sklearn.metrics.pairwise import cosine_similarity
import random
# コサイン類似度


def similarity(id1, id2):
    #a=[vectors[id2idx[id1]]]
    #b=[vectors[id2idx[id2]]]
    #return (dot(a, b) / (norm(a) * norm(b)))[0][0] 
    return cosine_similarity([vectors[id2idx[id1]]], [vectors[id2idx[id2]]])[0][0]


def make_triplets(dataset, threshold=0.85, seed=7, max_tries=25):
    triplets = []
    random.seed(seed)
    neg_candidate_indices = list(range(len(ids)))
    random.shuffle(neg_candidate_indices)

    def log(i, str):
        if i % 5000 == 0:
            print(str)

    for i, image_id in enumerate(list(dataset.keys())):
        log(i, "### %d ###" % (image_id))

        # pickup positive pair.
        score = 0.0
        tries = 0
        while score < threshold:
            #画像に付与したキャプションからランダムに2つ選んで類似度が閾値以上ならアンカーとポジティブにする
            [(id, caption), (id_pos, caption_pos)] = random.sample(dataset[image_id], 2)
            score = similarity(id, id_pos)
            tries += 1
            if tries > max_tries:
                break
        if score < threshold:
            continue
        
        #pickup negative one.
        id_neg=id
        current_caption_ids=[id_cap[0] for id_cap in dataset[image_id]]
        while id_neg in current_caption_ids:
            idx_neg=neg_candidate_indices.pop()
            id_neg=ids[idx_neg]
        caption_neg=captions[id2idx[id_neg]]
        
        log(i, "  pos:  score: %4.2f [%s]:[%s]" % (score, caption, caption_pos))
        log(i, "  neg:  score: %4.2f [%s]:[%s]" % (similarity(id, id_neg), caption, caption_neg))
        triplets.append({
          "image_id": image_id,
          "id": id,
          "id_pos": id_pos,
          "id_neg": id_neg,
          "caption": caption,
          "caption_pos": caption_pos,
          "caption_neg": caption_neg  
        })
    return triplets

In [ ]:
triplets = make_triplets(dataset)

### 580856 ###
  pos:  score: 0.86 [夜で、まだオープンしている場所の入り口]:[踏切の近くにワイナリーが開店している]
  neg:  score: 0.80 [夜で、まだオープンしている場所の入り口]:[野球選手のフォームの確認をしている]
### 235486 ###
  pos:  score: 0.91 [2人の女性と1人の男性が食卓を囲んでいる]:[三人の男女がピザと飲み物を囲んで笑っている]
  neg:  score: 0.85 [2人の女性と1人の男性が食卓を囲んでいる]:[ストックを使って子どもをひっぱってスキーをしている人]
### 133631 ###
  pos:  score: 0.97 [柵の手前に土の上に立った象が1頭いる]:[1頭のゾウが柵の内側に立っている]
  neg:  score: 0.78 [柵の手前に土の上に立った象が1頭いる]:[スケートボードに乗った少年がジャンプしている]
### 467843 ###
  pos:  score: 0.89 [メガネをかけた少年がフリスビーを見つめている]:[フリスビーで遊んでいる子どもだ]
  neg:  score: 0.91 [メガネをかけた少年がフリスビーを見つめている]:[黒い犬が芝生の上をフリスビーを加えながら走っている]
### 423588 ###
  pos:  score: 0.93 [雪山の小屋の屋根に登る人と撮影する人]:[雪山の小さな小屋の屋根に、スノーボードを履いた人物が乗り、それを3人の人物が見ている]
  neg:  score: 0.73 [雪山の小屋の屋根に登る人と撮影する人]:[とっても大きくてきれいな時計台がある]
### 115765 ###
  pos:  score: 0.92 [芝生の木立の中で番傘をさした女とギターケースを背負った男が話をしている]:[ギターを背負った男性と、民族衣装を着て傘をさした女性がいる]
  neg:  score: 0.84 [芝生の木立の中で番傘をさした女とギターケースを背負った男が話をしている]:[列車が停車している駅のホームに向かって線路を横切っている人がいる]
### 538330 ###
  pos:  score: 0.92 [マットの上に料理と飲み物が置いてあ

In [ ]:
#triplets = make_triplets(dataset)
#学習データの作成を行う
from sklearn.model_selection import train_test_split
train, dev_test = train_test_split(triplets, train_size=0.8, random_state=4)
dev, test = train_test_split(dev_test, train_size=0.5, random_state=7)

def to_tsv(fname, triplet):
  with open(fname, "w") as f:
    lines = ["%s\t%s\t%s" % (example["caption"], example["caption_pos"], example["caption_neg"]) for example in triplet]
    f.write("\n".join(lines)+"\n")

to_tsv("triplet_train.tsv", train)
to_tsv("triplet_dev.tsv", dev)
to_tsv("triplet_test.tsv", test)

del triplets
del ids
del captions
del dataset

In [ ]:
#パラメータ
BATCH_SIZE = 16
NUM_EPOCHS = 1
EVAL_STEPS = 1000
WARMUP_STEPS = int(len(train) // BATCH_SIZE * 0.1) 
OUTPUT_PATH = "./sbert_stair"

In [ ]:
import transformers
transformers.BertTokenizer = transformers.BertJapaneseTokenizer

from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from sentence_transformers.losses import TripletDistanceMetric, TripletLoss
from sentence_transformers.evaluation import TripletEvaluator
from sentence_transformers.readers import TripletReader
from sentence_transformers.datasets import SentencesDataset
from torch.utils.data import DataLoader

transformer = models.Transformer('cl-tohoku/bert-base-japanese-whole-word-masking')

pooling = models.Pooling(
    transformer.get_word_embedding_dimension(), 
    pooling_mode_mean_tokens=True, 
    pooling_mode_cls_token=False, 
    pooling_mode_max_tokens=False
)

model = SentenceTransformer(modules=[transformer, pooling])

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#データセットを読み込むDataLoaderを定義
triplet_reader=TripletReader(".")
train_data=SentencesDataset(triplet_reader.get_examples("triplet_train.tsv"),model=model)
train_dataloader=DataLoader(train_data,shuffle=True,batch_size=BATCH_SIZE)

In [ ]:
#ロスにはTripletLossを使う
train_loss = TripletLoss(model=model, distance_metric=TripletDistanceMetric.EUCLIDEAN, triplet_margin=1)

#検証もTripletで行う
dev_data = SentencesDataset(triplet_reader.get_examples('triplet_dev.tsv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=BATCH_SIZE)
#evaluator = TripletEvaluator(dev_dataloader)

In [ ]:
#学習ループの実行
model.fit(train_objectives=[(train_dataloader, train_loss)],
         epochs=NUM_EPOCHS,
         evaluation_steps=EVAL_STEPS,
         warmup_steps=WARMUP_STEPS,
         output_path=OUTPUT_PATH
         )

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6141 [00:00<?, ?it/s]